## MDM without Clustering

In [31]:
# Importing the required method.
from ragraph.io.csv import from_csv

# Converting csv files into a graph object.
g = from_csv(
    nodes_path="nodes.csv",
    edges_path="edges.csv",
    csv_delimiter=";",
    iter_delimiter=","  # Separates list elements within a cell.
)

import ragraph.plot

hardware = [n for n in g.get_nodes_by_kind("hardware")]
software = [n for n in g.get_nodes_by_kind("software")]
process = [n for n in g.get_nodes_by_kind("process")]


dsm = ragraph.plot.mdm(
    leafs = process + software + hardware,
    edges = g.edges,
    style= ragraph.plot.Style(
        piemap=dict(
            display="labels",
            mode="relative",
            fields=g.edge_labels
        ),
)
)



dsm

## MDM with Clustering

In [30]:
# Importing the required methods.
from ragraph.io.csv import from_csv
import ragraph.plot
import ragraph.analysis.cluster as cluster
import ragraph.analysis.sequence as seq

# 1. Converting csv files into a graph object.
g = from_csv(
    nodes_path="nodes.csv",
    edges_path="edges.csv",
    csv_delimiter=";",
    iter_delimiter=","
)

# Define the weight column
criticality_col = "criticality"

# 2. Define the domains
domain_kinds = ["process", "software", "hardware"]
final_ordered_leafs = []

# 3. Iterate through domains to Cluster and Sequence independently
for kind in domain_kinds:
    # Get the target nodes for this domain (e.g., all 'process' nodes)
    target_nodes = g.get_nodes_by_kind(kind)
    
    # Create a set for fast lookup of edges
    target_node_set = set(target_nodes)
    
    # --- FIX: Manually create a Subgraph ---
    # Create a new empty Graph of the same class as 'g'
    sg = g.__class__()
    
    # Add the specific nodes to this new graph
    # We check if bulk add exists, otherwise loop (makes it robust to version)
    if hasattr(sg, "add_nodes"):
        sg.add_nodes(target_nodes)
    else:
        for n in target_nodes:
            sg.add_node(n)
            
    # Find edges that connect ONLY these nodes
    # (We reuse the edge objects from the main graph)
    internal_edges = [
        e for e in g.edges 
        if e.source in target_node_set and e.target in target_node_set
    ]
    
    # Add these edges to the new graph
    if hasattr(sg, "add_edges"):
        sg.add_edges(internal_edges)
    else:
        for e in internal_edges:
            sg.add_edge(e)
    # ---------------------------------------

    # A. Clustering: Group nodes based on flow within this subgraph
    cluster.hierarchical_markov(sg, weights=criticality_col)
    
    # B. Sequencing: Reorder nodes to minimize feedback loops
    seq.genetic(sg, weights=criticality_col)
    
    # Collect the sorted leaves. 
    # Because we reused the original node objects, these correspond directly to 'g'
    final_ordered_leafs.extend(sg.leafs)

# 4. Generate the Plot
# We use the explicitly ordered list of leaves we just built.
dsm = ragraph.plot.mdm(
    leafs=final_ordered_leafs,
    edges=g.edges,
    style=ragraph.plot.Style(
        piemap=dict(
            display="labels",
            mode="relative",
            fields=g.edge_labels,
        ),
    ),
)

dsm

## MDM with Criticality

In [ ]:
from ragraph.analysis import heuristics
import ragraph
# Importing the required method.
from ragraph.io.csv import from_csv

# Converting csv files into a graph object.
g = from_csv(
    nodes_path="nodes.csv",
    edges_path="edges.csv",
    csv_delimiter=";",
    iter_delimiter=",",  # Separates list elements within a cell.
    edge_weights=["criticality"]
)

hardware = [n for n in g.get_nodes_by_kind("hardware")]
software = [n for n in g.get_nodes_by_kind("software")]
process = [n for n in g.get_nodes_by_kind("process")]

dsm = ragraph.plot.mdm(
    leafs = process + software + hardware,
    edges = g.edges,
    style= ragraph.plot.Style(
        piemap=dict(
            display="weights",   
            mode= "relative",
            fields=["criticality"]
        ),
    )
)

# Clustering the graph.
roots = heuristics.markov_gamma(
    graph=g,
    alpha=2,    # Expansion parameter.
    beta=2.0,   # Inflation parameter.
    mu=2.0,     # Evaporation parameter.
    gamma=2.0,   # Bus detection parameter.
    inplace=True
)

dsm

AttributeError: module 'ragraph' has no attribute 'plot'

## MDM Debug

In [27]:
# Importing the required method.
from ragraph.io.csv import from_csv

# Converting csv files into a graph object.
g = from_csv(
    nodes_path="nodes copy.csv",
    edges_path="edges copy.csv",
    csv_delimiter=";",
    iter_delimiter=","  # Separates list elements within a cell.
)

import ragraph.plot

hardware = [n for n in g.get_nodes_by_kind("hardware")]
software = [n for n in g.get_nodes_by_kind("software")]
process = [n for n in g.get_nodes_by_kind("process")]


dsm = ragraph.plot.mdm(
    leafs = process + software + hardware,
    edges = g.edges,
    style= ragraph.plot.Style(
        piemap=dict(
            display="labels",
            mode="relative",
            fields=g.edge_labels
        ),
)
)



dsm